In [1]:
import numpy as np
import pandas as pd
import math
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn import linear_model

# Initial Data


*   x1: Total Population *(in 1000s)*
*   x2: Population Density *(inhabitants/square km)*
*   x3: GDP per capita *(USD/inhabitant)*
*   x4: Long term average annual precipitation in depth *(mm/year)*
*   x5: Total renewable resources per capita *($m^{3}$/inhabitant /year)*
*   x6: Total water withdrawal per capita *($m^{3}$/inhabitant /year)*
*   x7: Agricultural water withdrawal *(%age of x6)*
*   x8: Industrial water withdrawal *(%age of x6)*
*   x9: Municipal water withdrawal *(%age of x6)*
*   x10: Collected Municipal wastewater  *($km^{3}$/year)*
*   x11: Capacity of the municipal wastewater treatment facilities *($km^{3}$/year)*
*   x12: Not treated municipal wastewater *($km^{3}$/year)*
*   y: Water Stress *(%age)*

In [2]:
#Read and prepare the data
data = pd.read_csv('https://www.dropbox.com/s/j07u99pg44kbfvm/final_data.csv?dl=1')
data

,Area,Variable Name,Year,Value
0,Afghanistan,Total population,1962,9351.441000
1,Afghanistan,Total population,1967,10399.926000
2,Afghanistan,Total population,1972,11791.215000
3,Afghanistan,Total population,1977,13171.306000
4,Afghanistan,Total population,1982,12882.528000
...,...,...,...,...
18698,Zimbabwe,SDG 6.4.2. Water Stress,2002,39.476155
18699,Zimbabwe,SDG 6.4.2. Water Stress,2007,33.514833
18700,Zimbabwe,SDG 6.4.2. Water Stress,2012,32.488030
18701,Zimbabwe,SDG 6.4.2. Water Stress,2017,31.346226


In [3]:
#data_x1_x5.pivot(columns='Variable Name', index='Area')['Value']
data = data.groupby(['Year','Variable Name', 'Area'])['Value'].aggregate('mean').unstack(1)
data

Variable Name                            Agricultural water withdrawal as % of total water withdrawal  \
Year Area                                                                                               
1962 Afghanistan                                                                       NaN              
     Albania                                                                           NaN              
     Algeria                                                                           NaN              
     Andorra                                                                           NaN              
     Angola                                                                            NaN              
...                                                                                    ...              
2018 Venezuela (Bolivarian Republic of)                                          73.852110              
     Viet Nam                                                                    94.782397              
     Yemen                                                                       90.743338              
     Zambia                                                                      73.282443              
     Zimbabwe                                                                    80.635470              

Variable Name                            Capacity of the municipal wastewater treatment facilities  \
Year Area                                                                                            
1962 Afghanistan                                                                       NaN           
     Albania                                                                           NaN           
     Algeria                                                                           NaN           
     Andorra                                                                           NaN           
     Angola                                                                            NaN           
...                                                                                    ...           
2018 Venezuela (Bolivarian Republic of)                                                NaN           
     Viet Nam                                                                        0.423           
     Yemen                                                                             NaN           
     Zambia                                                                            NaN           
     Zimbabwe                                                                          NaN           

Variable Name                            Collected municipal wastewater  \
Year Area                                                                 
1962 Afghanistan                                                    NaN   
     Albania                                                        NaN   
     Algeria                                                        NaN   
     Andorra                                                        NaN   
     Angola                                                         NaN   
...                                                                 ...   
2018 Venezuela (Bolivarian Republic of)                             NaN   
     Viet Nam                                                   0.19700   
     Yemen                                                          NaN   
     Zambia                                                         NaN   
     Zimbabwe                                                   0.00027   

Variable Name                            GDP per capita  \
Year Area                                                 
1962 Afghanistan                                    NaN   
     Albania                                        NaN   
     Algeria                                        NaN   
     Andorra                        

In [4]:
data.isna().sum()

Variable Name
Agricultural water withdrawal as % of total water withdrawal    1180
Capacity of the municipal wastewater treatment facilities       2206
Collected municipal wastewater                                  2170
GDP per capita                                                   409
Industrial water withdrawal as % of total water withdrawal      1204
Long-term average annual precipitation in volume                 210
Municipal water withdrawal as % of total withdrawal             1182
Not treated municipal wastewater                                2082
Population density                                                27
SDG 6.4.2. Water Stress                                         1104
Total population                                                  11
Total renewable water resources per capita                        41
Total water withdrawal per capita                               1090
dtype: int64

## Exploratory Data Analysis



*   Three variables, Capacity of the municipal wastewater treatment facilities (x11), Collected municipal wastewater (x10) and Not treated municipal wastewater (x12) are missing around 75% to 80% of the values. Therefore it can be assumed that they do not impact the water stress (y) values and can be droped from the dataset.
*   All the rows with NaN values can be dropped.
*   Variables x6, x7 and x8 sum up to 100 as they are percentages of x6. Therefore we need only two of these three variables and one of them should be dropped to avoid redundancy.
*   Total population and Population density are related variables. So one of them can be dropped. Total population being a more absolute variable can be dropped.





# Cleaning the dataset

In [5]:
#drop the rows that are missing "y" values i.e. missing SDG 6.4.2. Water Stress
print("Dimensions of the original dataset: ", data.shape)
data.dropna(subset=["Population density","GDP per capita","SDG 6.4.2. Water Stress","Agricultural water withdrawal as % of total water withdrawal","Municipal water withdrawal as % of total withdrawal"], inplace=True)
data.drop(["Total population","Capacity of the municipal wastewater treatment facilities","Collected municipal wastewater","Not treated municipal wastewater","Industrial water withdrawal as % of total water withdrawal"], axis=1, inplace=True)
print("Dimensions of the new dataset: ", data.shape)

Dimensions of the original dataset:  (2431, 13)
Dimensions of the new dataset:  (1205, 8)


In [6]:
data.isna().sum()

Variable Name
Agricultural water withdrawal as % of total water withdrawal    0
GDP per capita                                                  0
Long-term average annual precipitation in volume                0
Municipal water withdrawal as % of total withdrawal             0
Population density                                              0
SDG 6.4.2. Water Stress                                         0
Total renewable water resources per capita                      0
Total water withdrawal per capita                               0
dtype: int64

In [7]:
data.reset_index(inplace=True)

In [8]:
data

Variable Name,Year,Area,Agricultural water withdrawal as % of total water withdrawal,GDP per capita,Long-term average annual precipitation in volume,Municipal water withdrawal as % of total withdrawal,Population density,SDG 6.4.2. Water Stress,Total renewable water resources per capita,Total water withdrawal per capita
0,1972,Denmark,31.947484,4655.290175,30.29227,60.831510,115.816802,24.716063,1202.273258,183.146293
1,1972,Ghana,54.727904,493.336524,283.14698,32.902354,38.692173,1.505958,6089.084168,37.430171
2,1972,Hungary,22.379496,782.184870,54.79467,20.569268,112.072299,5.828256,9974.980064,323.626719
3,1972,Israel,78.318250,2762.062183,9.60045,16.779789,135.771726,135.122077,594.029932,531.022713
4,1972,Italy,59.201104,2674.044383,250.71488,16.811125,180.058210,37.599562,3525.699205,786.449189
...,...,...,...,...,...,...,...,...,...,...
1200,2018,Venezuela (Bolivarian Republic of),73.852110,6433.928949,1864.23020,22.641793,31.672735,7.540367,45868.196340,783.266091
1201,2018,Viet Nam,94.782397,2538.068147,603.16983,1.470194,288.458056,18.130315,9253.347619,858.539684
1202,2018,Yemen,90.743338,922.404203,88.17099,7.433380,53.977845,169.761905,73.687616,125.093500
1203,2018,Zambia,73.282443,1549.354235,767.66220,18.447837,23.055378,2.835498,6039.751245,90.596269


In [9]:
data.corr()

Variable Name,Year,Agricultural water withdrawal as % of total water withdrawal,GDP per capita,Long-term average annual precipitation in volume,Municipal water withdrawal as % of total withdrawal,Population density,SDG 6.4.2. Water Stress,Total renewable water resources per capita,Total water withdrawal per capita
Variable Name,,,,,,,,,
Year,1.000000,-0.060807,0.223303,0.024027,0.078427,0.023163,0.030757,-0.068900,-0.051328
Agricultural water withdrawal as % of total water withdrawal,-0.060807,1.000000,-0.415532,0.018947,-0.675577,-0.180608,0.085287,-0.089993,0.296968
GDP per capita,0.223303,-0.415532,1.000000,0.026815,0.250539,0.165558,0.170544,0.093298,0.035759
Long-term average annual precipitation in volume,0.024027,0.018947,0.026815,1.000000,-0.094149,-0.073010,-0.066559,0.054745,0.066076
Municipal water withdrawal as % of total withdrawal,0.078427,-0.675577,0.250539,-0.094149,1.000000,0.148729,-0.011202,0.099943,-0.422910
Population density,0.023163,-0.180608,0.165558,-0.073010,0.148729,1.000000,0.045586,-0.090589,-0.069448
SDG 6.4.2. Water Stress,0.030757,0.085287,0.170544,-0.066559,-0.011202,0.045586,1.000000,-0.078604,0.079976
Total renewable water resources per capita,-0.068900,-0.089993,0.093298,0.054745,0.099943,-0.090589,-0.078604,1.000000,0.094333
Total water withdrawal per capita,-0.051328,0.296968,0.035759,0.066076,-0.422910,-0.069448,0.079976,0.094333,1.000000


There is no direct linear relation between any of the variables and the water stress values. We need to build a prediction model that analyze complex relations between variables such as Neural Networks.

# Scaling the dataset

In [10]:
#Create X_data and Y_data
X_data= data[["Agricultural water withdrawal as % of total water withdrawal", "GDP per capita", "Long-term average annual precipitation in volume", "Municipal water withdrawal as % of total withdrawal", "Total renewable water resources per capita", "Total water withdrawal per capita", "Population density"]]
Y_data= data["SDG 6.4.2. Water Stress"]
X_scale= X_data.apply(lambda x: (x-x.min(axis=0)) / (x.max(axis=0)-x.min(axis=0)))
#Change the percentage to proportion
Y_scale=Y_data.apply(lambda x: x/100)

# Neural Network Model

## Splitting the dataset

In [11]:
print(X_scale.shape,Y_scale.shape)

(1205, 7) (1205,)


In [12]:
#convert dataframes to array
X=pd.DataFrame(X_scale).to_numpy()
Y=pd.DataFrame(Y_scale).to_numpy()

In [13]:
X

array([[3.20902724e-01, 3.84416071e-02, 2.01606512e-03, ...,
        1.64310175e-03, 3.06699357e-02, 1.43787686e-02],
       [5.49725098e-01, 3.78738908e-03, 1.88773038e-02, ...,
        8.34868443e-03, 5.25083958e-03, 4.67859864e-03],
       [2.24795207e-01, 6.19246477e-03, 3.64997106e-03, ...,
        1.36808319e-02, 5.51757034e-02, 1.39078125e-02],
       ...,
       [9.11489140e-01, 7.35999131e-03, 5.87562111e-03, ...,
        9.44794555e-05, 2.05430567e-02, 6.60111833e-03],
       [7.36099775e-01, 1.25802472e-02, 5.11864775e-02, ...,
        8.28099080e-03, 1.45252708e-02, 2.71191726e-03],
       [8.09958693e-01, 1.16482843e-02, 1.71156816e-02, ...,
        1.89405038e-03, 4.42854017e-02, 4.45955169e-03]])

In [14]:
Y

array([[0.24716063],
       [0.01505958],
       [0.05828256],
       ...,
       [1.69761905],
       [0.02835498],
       [0.3540537 ]])

In [15]:
#split the data for testing and training
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(964, 7) (241, 7) (964, 1) (241, 1)


## Building the Neural Network Model

In [16]:
#Create a neutral network model
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten()),
model.add(tf.keras.layers.Dense(35, activation="relu")),
model.add(tf.keras.layers.Dense(30, activation="sigmoid")),
model.add(tf.keras.layers.Dense(25, activation="sigmoid")),
model.add(tf.keras.layers.Dense(1, activation="linear")),

#compile the model
model.compile(optimizer="adam",loss="mean_squared_error", metrics="accuracy")

In [17]:
#train the model
model.fit(X_train, Y_train, epochs=450, batch_size=32)

Epoch 1/450
31/31 [==============================] - 1s 3ms/step - loss: 8.9759 - accuracy: 0.0000e+00
Epoch 2/450
31/31 [==============================] - 0s 2ms/step - loss: 7.8904 - accuracy: 0.0000e+00
Epoch 3/450
31/31 [==============================] - 0s 2ms/step - loss: 7.7503 - accuracy: 0.0000e+00
Epoch 4/450
31/31 [==============================] - 0s 2ms/step - loss: 7.7306 - accuracy: 0.0000e+00
Epoch 5/450
31/31 [==============================] - 0s 2ms/step - loss: 7.7277 - accuracy: 0.0000e+00
Epoch 6/450
31/31 [==============================] - 0s 2ms/step - loss: 7.7155 - accuracy: 0.0000e+00
Epoch 7/450
31/31 [==============================] - 0s 2ms/step - loss: 7.7124 - accuracy: 0.0000e+00
Epoch 8/450
31/31 [==============================] - 0s 2ms/step - loss: 7.6917 - accuracy: 0.0000e+00
Epoch 9/450
31/31 [==============================] - 0s 2ms/step - loss: 7.6735 - accuracy: 0.0000e+00
Epoch 10/450
31/31 [==============================] - 0s 2ms/step - loss:

31/31 [==============================] - 0s 2ms/step - loss: 4.7272 - accuracy: 0.0000e+00
Epoch 80/450
31/31 [==============================] - 0s 2ms/step - loss: 4.6903 - accuracy: 0.0000e+00
Epoch 81/450
31/31 [==============================] - 0s 2ms/step - loss: 4.6523 - accuracy: 0.0000e+00
Epoch 82/450
31/31 [==============================] - 0s 2ms/step - loss: 4.6297 - accuracy: 0.0000e+00
Epoch 83/450
31/31 [==============================] - 0s 3ms/step - loss: 4.5778 - accuracy: 0.0000e+00
Epoch 84/450
31/31 [==============================] - 0s 2ms/step - loss: 4.5521 - accuracy: 0.0000e+00
Epoch 85/450
31/31 [==============================] - 0s 3ms/step - loss: 4.5885 - accuracy: 0.0000e+00
Epoch 86/450
31/31 [==============================] - 0s 3ms/step - loss: 4.5176 - accuracy: 0.0000e+00
Epoch 87/450
31/31 [==============================] - 0s 2ms/step - loss: 4.4849 - accuracy: 0.0000e+00
Epoch 88/450
31/31 [==============================] - 0s 3ms/step - loss: 4.4

31/31 [==============================] - 0s 2ms/step - loss: 3.2122 - accuracy: 0.0000e+00
Epoch 158/450
31/31 [==============================] - 0s 2ms/step - loss: 3.3233 - accuracy: 0.0000e+00
Epoch 159/450
31/31 [==============================] - 0s 2ms/step - loss: 3.2558 - accuracy: 0.0000e+00
Epoch 160/450
31/31 [==============================] - 0s 2ms/step - loss: 3.2260 - accuracy: 0.0000e+00
Epoch 161/450
31/31 [==============================] - 0s 2ms/step - loss: 3.2072 - accuracy: 0.0000e+00
Epoch 162/450
31/31 [==============================] - 0s 2ms/step - loss: 3.3137 - accuracy: 0.0000e+00
Epoch 163/450
31/31 [==============================] - 0s 2ms/step - loss: 3.1678 - accuracy: 0.0000e+00
Epoch 164/450
31/31 [==============================] - 0s 2ms/step - loss: 3.1877 - accuracy: 0.0000e+00
Epoch 165/450
31/31 [==============================] - 0s 2ms/step - loss: 3.1479 - accuracy: 0.0000e+00
Epoch 166/450
31/31 [==============================] - 0s 2ms/step - 

31/31 [==============================] - 0s 2ms/step - loss: 2.8176 - accuracy: 0.0000e+00
Epoch 236/450
31/31 [==============================] - 0s 2ms/step - loss: 2.7962 - accuracy: 0.0000e+00
Epoch 237/450
31/31 [==============================] - 0s 2ms/step - loss: 2.6791 - accuracy: 0.0000e+00
Epoch 238/450
31/31 [==============================] - 0s 2ms/step - loss: 2.6783 - accuracy: 0.0000e+00
Epoch 239/450
31/31 [==============================] - 0s 2ms/step - loss: 2.6836 - accuracy: 0.0000e+00
Epoch 240/450
31/31 [==============================] - 0s 2ms/step - loss: 2.6820 - accuracy: 0.0000e+00
Epoch 241/450
31/31 [==============================] - 0s 2ms/step - loss: 2.6366 - accuracy: 0.0000e+00
Epoch 242/450
31/31 [==============================] - 0s 2ms/step - loss: 2.6795 - accuracy: 0.0000e+00
Epoch 243/450
31/31 [==============================] - 0s 2ms/step - loss: 2.7908 - accuracy: 0.0000e+00
Epoch 244/450
31/31 [==============================] - 0s 2ms/step - 

31/31 [==============================] - 0s 2ms/step - loss: 2.2957 - accuracy: 0.0000e+00
Epoch 314/450
31/31 [==============================] - 0s 2ms/step - loss: 2.3562 - accuracy: 0.0000e+00
Epoch 315/450
31/31 [==============================] - 0s 2ms/step - loss: 2.2816 - accuracy: 0.0000e+00
Epoch 316/450
31/31 [==============================] - 0s 2ms/step - loss: 2.4042 - accuracy: 0.0000e+00
Epoch 317/450
31/31 [==============================] - 0s 2ms/step - loss: 2.5988 - accuracy: 0.0000e+00
Epoch 318/450
31/31 [==============================] - 0s 2ms/step - loss: 2.2667 - accuracy: 0.0000e+00
Epoch 319/450
31/31 [==============================] - 0s 2ms/step - loss: 2.3319 - accuracy: 0.0000e+00
Epoch 320/450
31/31 [==============================] - 0s 2ms/step - loss: 2.3390 - accuracy: 0.0000e+00
Epoch 321/450
31/31 [==============================] - 0s 2ms/step - loss: 2.5042 - accuracy: 0.0000e+00
Epoch 322/450
31/31 [==============================] - 0s 2ms/step - 

31/31 [==============================] - 0s 3ms/step - loss: 2.0198 - accuracy: 0.0000e+00
Epoch 392/450
31/31 [==============================] - 0s 3ms/step - loss: 2.1218 - accuracy: 0.0000e+00
Epoch 393/450
31/31 [==============================] - 0s 3ms/step - loss: 2.1359 - accuracy: 0.0000e+00
Epoch 394/450
31/31 [==============================] - 0s 4ms/step - loss: 2.2039 - accuracy: 0.0000e+00
Epoch 395/450
31/31 [==============================] - 0s 3ms/step - loss: 2.2604 - accuracy: 0.0000e+00
Epoch 396/450
31/31 [==============================] - 0s 2ms/step - loss: 1.9777 - accuracy: 0.0000e+00
Epoch 397/450
31/31 [==============================] - 0s 3ms/step - loss: 2.0531 - accuracy: 0.0000e+00
Epoch 398/450
31/31 [==============================] - 0s 3ms/step - loss: 2.1100 - accuracy: 0.0000e+00
Epoch 399/450
31/31 [==============================] - 0s 3ms/step - loss: 2.0145 - accuracy: 0.0000e+00
Epoch 400/450
31/31 [==============================] - 0s 3ms/step - 

## Predicting the test data

In [18]:
Y_hat=model.predict(X_test)
Y_hat

8/8 [==============================] - 0s 1ms/step


array([[ 0.7179318 ],
       [ 0.14259958],
       [ 0.6663942 ],
       [ 0.07807833],
       [ 0.08353722],
       [ 0.07509935],
       [ 0.19101822],
       [ 0.16766697],
       [ 0.07509935],
       [ 0.20958626],
       [ 0.39574462],
       [ 0.14912128],
       [ 0.21232665],
       [ 0.33833754],
       [ 0.8213795 ],
       [ 0.07871932],
       [ 0.08050418],
       [ 0.23019129],
       [ 0.09562194],
       [ 0.34592122],
       [ 0.16889066],
       [ 0.07509959],
       [ 0.1844911 ],
       [ 0.2929303 ],
       [ 0.40083086],
       [ 0.37147468],
       [ 0.56945115],
       [ 0.23851901],
       [ 0.07509935],
       [ 0.19429195],
       [ 0.32545173],
       [ 0.07625467],
       [ 0.46853036],
       [ 0.07721889],
       [ 0.19393581],
       [ 0.09648085],
       [ 0.4816519 ],
       [ 0.14166659],
       [ 0.4939857 ],
       [ 0.10705298],
       [ 0.568876  ],
       [ 0.1018458 ],
       [ 5.967721  ],
       [ 0.07520735],
       [ 0.07974213],
       [ 0

In [19]:
Y_test

array([[1.16714286e+00],
       [8.17057212e-02],
       [7.87544531e-01],
       [1.49572650e-02],
       [5.91313131e-02],
       [2.47898551e-03],
       [6.27029448e-02],
       [5.08756567e-02],
       [5.02166065e-03],
       [5.73452485e-02],
       [5.48754749e-01],
       [5.74209708e-02],
       [1.40845753e-01],
       [2.71312265e-01],
       [9.10891089e-01],
       [3.43948718e-02],
       [6.78235294e-02],
       [2.14908880e-01],
       [6.63979132e-02],
       [1.74084416e-01],
       [5.08756567e-02],
       [2.30433000e-02],
       [1.35503388e-01],
       [8.46153846e-02],
       [6.84512428e-02],
       [3.67736151e-01],
       [6.41022433e-01],
       [2.07942170e-01],
       [1.32242833e-03],
       [1.75048381e-01],
       [4.02076574e-01],
       [3.35648148e-03],
       [1.01250000e+00],
       [3.03260459e-02],
       [3.39163498e-02],
       [3.39500734e-02],
       [6.20557157e-01],
       [1.94195387e-01],
       [1.43000000e-01],
       [3.46646571e-02],


Problems to be addressed:
*   Analyze the performance of the model.
*   Prediction of coutries that will run out of water by 2050. (can be verified by checking if the Water stress will reach 100% or 1.0)
*   No justification behind chossing the certain number of hidden layers and certain number of neurons.
*   Further use this model to predict values of water stress in the app.











# Predict the x variables for future

In [20]:
#Read and prepare the data
data_ori = pd.read_csv('https://www.dropbox.com/s/j07u99pg44kbfvm/final_data.csv?dl=1')
data_ori

,Area,Variable Name,Year,Value
0,Afghanistan,Total population,1962,9351.441000
1,Afghanistan,Total population,1967,10399.926000
2,Afghanistan,Total population,1972,11791.215000
3,Afghanistan,Total population,1977,13171.306000
4,Afghanistan,Total population,1982,12882.528000
...,...,...,...,...
18698,Zimbabwe,SDG 6.4.2. Water Stress,2002,39.476155
18699,Zimbabwe,SDG 6.4.2. Water Stress,2007,33.514833
18700,Zimbabwe,SDG 6.4.2. Water Stress,2012,32.488030
18701,Zimbabwe,SDG 6.4.2. Water Stress,2017,31.346226


In [21]:
data_ori = data_ori.groupby(['Year','Variable Name', 'Area'])['Value'].aggregate('mean').unstack(1)
data_ori.reset_index(inplace=True)
print("Dimensions of the original dataset: ", data_ori.shape)
data_ori.dropna(subset=["Population density","GDP per capita","SDG 6.4.2. Water Stress","Agricultural water withdrawal as % of total water withdrawal","Municipal water withdrawal as % of total withdrawal"], inplace=True)
data_ori.drop(["Total population","Capacity of the municipal wastewater treatment facilities","Collected municipal wastewater","Not treated municipal wastewater","Industrial water withdrawal as % of total water withdrawal"], axis=1, inplace=True)
print("Dimensions of the new dataset: ", data_ori.shape)
data_ori


Dimensions of the original dataset:  (2431, 15)
Dimensions of the new dataset:  (1205, 10)


Variable Name,Year,Area,Agricultural water withdrawal as % of total water withdrawal,GDP per capita,Long-term average annual precipitation in volume,Municipal water withdrawal as % of total withdrawal,Population density,SDG 6.4.2. Water Stress,Total renewable water resources per capita,Total water withdrawal per capita
388,1972,Denmark,31.947484,4655.290175,30.29227,60.831510,115.816802,24.716063,1202.273258,183.146293
404,1972,Ghana,54.727904,493.336524,283.14698,32.902354,38.692173,1.505958,6089.084168,37.430171
414,1972,Hungary,22.379496,782.184870,54.79467,20.569268,112.072299,5.828256,9974.980064,323.626719
421,1972,Israel,78.318250,2762.062183,9.60045,16.779789,135.771726,135.122077,594.029932,531.022713
422,1972,Italy,59.201104,2674.044383,250.71488,16.811125,180.058210,37.599562,3525.699205,786.449189
...,...,...,...,...,...,...,...,...,...,...
2426,2018,Venezuela (Bolivarian Republic of),73.852110,6433.928949,1864.23020,22.641793,31.672735,7.540367,45868.196340,783.266091
2427,2018,Viet Nam,94.782397,2538.068147,603.16983,1.470194,288.458056,18.130315,9253.347619,858.539684
2428,2018,Yemen,90.743338,922.404203,88.17099,7.433380,53.977845,169.761905,73.687616,125.093500
2429,2018,Zambia,73.282443,1549.354235,767.66220,18.447837,23.055378,2.835498,6039.751245,90.596269


In [22]:
def extrapolate(country,variable):
  poly = np.polyfit(data_ori[data_ori['Area']==country]['Year'], data_ori[data_ori['Area']==country][variable], deg=1)
  y_int  = np.polyval(poly, 2022)
  return y_int

In [23]:
newdata = data_ori[data_ori['Area']=='country']

In [24]:
ind=0
for i in np.unique(np.array(data_ori['Area'])):
  #print(ind)
  #print(i)
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
                                'Agricultural water withdrawal as % of total water withdrawal':extrapolate(i,'Agricultural water withdrawal as % of total water withdrawal'),
                                'GDP per capita':extrapolate(i,'GDP per capita'), 
                                'Long-term average annual precipitation in volume':extrapolate(i,'Long-term average annual precipitation in volume'),
                                'Municipal water withdrawal as % of total withdrawal':extrapolate(i,'Municipal water withdrawal as % of total withdrawal'),
                                'Population density':extrapolate(i,'Population density'), 
                                'SDG 6.4.2. Water Stress':extrapolate(i,'SDG 6.4.2. Water Stress'),
                                'Total renewable water resources per capita':extrapolate(i,'Total renewable water resources per capita'),
                                'Total water withdrawal per capita':extrapolate(i,'Total water withdrawal per capita')})
  #print(newdata.loc[ind])
  ind=ind+1

C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata.loc[ind] = pd.Series({'Year':2022, 'Area':i,
C:\Users\nikte\AppData\Local\Temp\ipykernel_5752\1285327992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [25]:
newdata

Variable Name,Year,Area,Agricultural water withdrawal as % of total water withdrawal,GDP per capita,Long-term average annual precipitation in volume,Municipal water withdrawal as % of total withdrawal,Population density,SDG 6.4.2. Water Stress,Total renewable water resources per capita,Total water withdrawal per capita
0,2022,Afghanistan,97.941104,656.180969,213.48522,0.993969,61.257508,50.562994,969.975594,176.892287
1,2022,Albania,57.542793,5786.599868,41.381326,30.362635,101.094659,6.154082,10689.844269,363.92748
2,2022,Algeria,58.977252,4659.461837,211.974923,38.004788,18.40922,142.115966,233.342039,255.148373
3,2022,Angola,5.890727,4653.147601,1259.167,53.707643,25.809087,2.03677,3238.309847,21.052376
4,2022,Antigua and Barbuda,14.236588,16491.180458,0.4532,63.205424,231.527633,9.855674,485.886409,131.255451
...,...,...,...,...,...,...,...,...,...,...
172,2022,Venezuela (Bolivarian Republic of),76.424646,9129.766093,1864.069566,21.187347,35.293496,8.25744,32080.107666,819.597845
173,2022,Viet Nam,95.87318,2400.104517,602.640497,0.828757,304.226958,20.510049,8166.541004,944.226983
174,2022,Yemen,90.544375,1217.420441,88.17099,7.490926,57.992638,177.354961,56.084185,106.871156
175,2022,Zambia,72.242013,1861.535819,767.6622,18.899592,24.287317,2.745366,4918.65272,67.947275


In [26]:
newdata.to_csv('validation_data.csv', encoding = 'utf-8-sig') 